# Accessing Free Hourly Weather Data
NOAA stores Free Hourly Weather Data at: ftp://ftp.ncdc.noaa.gov/pub/data/noaa/

It is my understanding that this is the data underlying most of the websites that charge for historical weather data at the hourly level (at least when needed in bulk).

The most detailed data is in a very cumbersome format, but a subset of easy to parse data can be found at: ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/

To make this topic approachable to a large audience, I've added some notes on how one could access this data simply using Excel at the end of this notebook.

In [138]:
# Column Names were determined from ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/isd-lite-format.pdf
# That pdf describes what data is contained in the subset of data that I'll focus on.
isd_fwf_cols = ['year', 'month', 'day', 'hour', 'air_temp_c', 'dew_pt_temp_c',
                 'sea_lvl_press_hectoPa', 'wnd_dir_360', 'wnd_spd_mtrpersec',
                 'sky_condition', 'precip_hrly', 'precip_6hr_accum']

In [139]:
# Importing the python libraries that I use.
import pandas as pd
import numpy as np

In [140]:
# Importing the table defining the available data. 
# There is a row for each station and it includes the begin and end date of available data.
isd_stations_data = pd.read_csv('ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.csv')
isd_stations_data.head()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,NaN,NaN,NaN,0.00,0.000,7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,NaN,NaN,0.00,0.000,7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,NaN,NaN,0.00,0.000,7070.0,20140923,20150926
3,008260,99999,WXPOD8270,NaN,NaN,NaN,0.00,0.000,0.0,20050101,20100731
4,008268,99999,WXPOD8278,AF,NaN,NaN,32.95,65.567,1156.7,20100519,20120323


In [141]:
# I want data for DC, so I've chosen to search for the local airport. Reagan National Airport (DCA).
# Note that all of the Station Names are uppercase.
DCA_search = isd_stations_data.loc[(isd_stations_data['STATION NAME'].isna() == False) 
                                   & (isd_stations_data['STATION NAME'].str.contains('REAGAN'))]

In [142]:
# Slicing out the BEGIN and END years to create the range of years for which I'll download data.
start_year = str(list(DCA_search.BEGIN)[0])[0:4]
end_year = str(list(DCA_search.END)[0])[0:4]
year_range = range(int(start_year), int(end_year)+1)
year_range

range(1936, 2020)

In [143]:
# Creating the station ID by which the ftp site is organized.
# Note that it is the concatenation of two columns separated by a hyphen.
station_id = str(list(DCA_search.USAF)[0])+'-'+str(list(DCA_search.WBAN)[0])
station_id

'724050-13743'

In [144]:
# Function to loop through a given station ID for a given range of years.
def download_isd_lite(station_id, year_range):
    isd_df = pd.DataFrame()
    for year in year_range:
        # There can be gaps of missing years in the data, so try and except were required. 
        # The gaps that I've seen are only from decades ago.
        try:
            new_isd_df = pd.read_fwf('ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/'
                                     +str(year)+'/'
                                     +station_id+'-'
                                     +str(year)
                                     +'.gz',
                                     header=None)
            isd_df = pd.concat([isd_df, new_isd_df])
        except:
            continue
    
    # Resetting the index of the concatenated DataFrame
    isd_df.reset_index(inplace=True, drop=True)
    
    # Setting the column names that I've derived from the format guide
    isd_df.columns = isd_fwf_cols
   
    # NOAA populates missing values with -9999, but I've chosen to replace them with NaN's.
    isd_df.replace({-9999: np.nan}, inplace=True)
    
    # Some of the columns are scaled by a factor of 10 to eliminate decimal points,
    # which would complicate the fixed width format that NOAA has chosen to utilize
    scaled_columns = ['air_temp_c', 'dew_pt_temp_c', 'sea_lvl_press_hectoPa', 
                  'wnd_spd_mtrpersec', 'precip_hrly', 'precip_6hr_accum']
    scaling_factor = 10
    # Resolving the scaling factor
    isd_df[scaled_columns] = isd_df[scaled_columns] / 10
    
    # Creating a date_time column from the various time-based columns NOAA provides.
    # The first step is creating a properly formatted string that pandas can parse, and then parse them.
    isd_df['date_time'] = isd_df.day.astype('int').astype('str').str.zfill(2)+'/'\
                         +isd_df.month.astype('int').astype('str').str.zfill(2)+'/'\
                         +isd_df.year.astype('int').astype('str')+'/'\
                         +isd_df.hour.astype('int').astype('str').str.zfill(2)
    isd_df['date_time'] = pd.to_datetime(isd_df['date_time'], format='%d/%m/%Y/%H')
    
    return isd_df

In [145]:
# Running the function for DCA for all years
isd_df = download_isd_lite(station_id, year_range)

In [146]:
# Inspecting the results
isd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564220 entries, 0 to 564219
Data columns (total 13 columns):
year                     564220 non-null float64
month                    564220 non-null float64
day                      564220 non-null float64
hour                     564220 non-null float64
air_temp_c               564215 non-null float64
dew_pt_temp_c            562838 non-null float64
sea_lvl_press_hectoPa    502737 non-null float64
wnd_dir_360              562159 non-null float64
wnd_spd_mtrpersec        564167 non-null float64
sky_condition            462316 non-null float64
precip_hrly              468604 non-null float64
precip_6hr_accum         35259 non-null float64
date_time                564220 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(12)
memory usage: 56.0 MB


In [147]:
isd_df.tail()

,year,month,day,hour,air_temp_c,dew_pt_temp_c,sea_lvl_press_hectoPa,wnd_dir_360,wnd_spd_mtrpersec,sky_condition,precip_hrly,precip_6hr_accum,date_time
564215,2019.0,9.0,24.0,3.0,26.1,18.9,1010.8,310.0,5.7,NaN,0.0,NaN,2019-09-24 03:00:00
564216,2019.0,9.0,24.0,4.0,26.1,18.3,1010.5,300.0,4.6,NaN,0.0,NaN,2019-09-24 04:00:00
564217,2019.0,9.0,24.0,5.0,25.6,17.8,1010.6,300.0,3.6,NaN,0.0,NaN,2019-09-24 05:00:00
564218,2019.0,9.0,24.0,6.0,24.4,17.8,1010.8,310.0,4.6,NaN,NaN,NaN,2019-09-24 06:00:00
564219,2019.0,9.0,24.0,7.0,23.9,17.8,1010.6,330.0,4.1,NaN,NaN,NaN,2019-09-24 07:00:00


In [148]:
# Not much missing temperature data, and the mising values aren't recent.
isd_df.loc[isd_df.air_temp_c.isna()].tail()

,year,month,day,hour,air_temp_c,dew_pt_temp_c,sea_lvl_press_hectoPa,wnd_dir_360,wnd_spd_mtrpersec,sky_condition,precip_hrly,precip_6hr_accum,date_time
287199,1988.0,1.0,4.0,8.0,NaN,NaN,NaN,340.0,3.1,8.0,1.3,NaN,1988-01-04 08:00:00
290306,1988.0,5.0,17.0,5.0,NaN,NaN,NaN,999.0,99.9,NaN,0.0,NaN,1988-05-17 05:00:00
290576,1988.0,5.0,28.0,17.0,NaN,11.7,1020.3,180.0,1.5,NaN,0.0,0.0,1988-05-28 17:00:00
291813,1988.0,7.0,20.0,22.0,NaN,NaN,NaN,310.0,10.8,8.0,9.9,NaN,1988-07-20 22:00:00
291830,1988.0,7.0,21.0,17.0,NaN,NaN,NaN,999.0,99.9,NaN,0.0,NaN,1988-07-21 17:00:00


In [149]:
# More missing humidity, but not recently.
isd_df.loc[isd_df.dew_pt_temp_c.isna()].tail()

,year,month,day,hour,air_temp_c,dew_pt_temp_c,sea_lvl_press_hectoPa,wnd_dir_360,wnd_spd_mtrpersec,sky_condition,precip_hrly,precip_6hr_accum,date_time
291813,1988.0,7.0,20.0,22.0,NaN,NaN,NaN,310.0,10.8,8.0,9.9,NaN,1988-07-20 22:00:00
291830,1988.0,7.0,21.0,17.0,NaN,NaN,NaN,999.0,99.9,NaN,0.0,NaN,1988-07-21 17:00:00
292024,1988.0,7.0,30.0,0.0,33.3,NaN,1019.6,270.0,3.6,0.0,0.0,0.0,1988-07-30 00:00:00
292026,1988.0,7.0,30.0,2.0,33.3,NaN,1019.6,270.0,3.6,0.0,0.0,0.0,1988-07-30 02:00:00
293337,1988.0,9.0,24.0,10.0,13.3,NaN,1014.9,30.0,6.2,8.0,0.0,NaN,1988-09-24 10:00:00


# Excel Users
You could create the ftp URLs in Excel and then manually click all the links you create. If you had a column with the station_id and a column with the year for each file you want then you could construct the URLs as follows:

=HYPERLINK("ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-lite/"&reference_year_cell&"/"&reference_station_id_cell&"-"&reference_year_cell&".gz")

You can then unzip each file, and open it in Excel. Using Excel's "Text to Columns" feature with the "Original data type" option set to "Fixed width", Excel will correctly separate the data in to columns. You can then manually add the column headers and save the data as an Excel file as desired. Then you could manually aggregate the data for multiple years and/or stations as needed.

# Mapping the 1000 Active US Airport Weather Stations with the Longest Recorded History

The map created below shows active (defined as having reported weather data in the last 2 days) weather stations at US airports. To ensure the map could be rendered, only 1000 weather stations were selected and those 1000 were selected based on having the oldest beginning date of data availability.

In [150]:
import folium

In [151]:
# Removing weather stations without valid coordinates
stations_with_coordinates = isd_stations_data.loc[((isd_stations_data.LAT != 0) 
                                                  & (isd_stations_data.LON != 0))
                                                  & (isd_stations_data.LAT.isna() == False)
                                                  & (isd_stations_data.LON.isna() == False)].copy()

In [152]:
# creating a list of the latitude and longitude coordinates, which will be fed into the mapping function
stations_with_coordinates['COORDINATES'] = list(zip(round(stations_with_coordinates.LAT,3), 
                                                    round(stations_with_coordinates.LON,3)))

In [153]:
# Trying to reduce the volume so a reasonable map can be rendered, and only active sites are included
most_recent_day = stations_with_coordinates['END'].max()
us_airports_with_coordinates = stations_with_coordinates.loc[(stations_with_coordinates.CTRY == 'US')
                                                             & (stations_with_coordinates.ICAO.isna() == False)
                                                             & (stations_with_coordinates.END > (most_recent_day-2))].copy()

In [154]:
# creating a string with the plain language Station Name and the Station ID used in the ftp site, 
# which will be presented on the map
us_airports_with_coordinates['IDENTIFIER'] = (us_airports_with_coordinates['STATION NAME']+'; Station ID: '
                                              +us_airports_with_coordinates['USAF'].astype('str')+'-'
                                              +us_airports_with_coordinates['WBAN'].astype('str'))

In [155]:
# sorting data so that when I trim the number of entries using .head(), I preserve those with the longest history of data
us_airports_with_coordinates.sort_values(by=['BEGIN'], ascending=True, inplace=True)

In [156]:
# Creating a map of the selected airport weather stations with annotations
base_map = folium.Map([52, -112], zoom_start=3)
# Limiting the map to 1000 locations
for index, coord in enumerate(us_airports_with_coordinates['COORDINATES'].head(1000)):
    marker = folium.Marker([coord[0],coord[1]], popup=us_airports_with_coordinates.iloc[index]['IDENTIFIER']).add_to(base_map)

In [157]:
# Here's the interactive map that you can use to find the Station ID for a given US location.
base_map

In [158]:
base_map.save('1000_US_Airports_Weather_Stations_Map.html')